<a href="https://colab.research.google.com/github/antocommi/provagcp/blob/master/Dashboard_sporco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
SEED = 9126

import os, cv2, json, time, math, sys, pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

from PIL import Image
import pandas as pd
import numpy as np
np.random.seed(SEED)
import seaborn as sn

#from vmz import models
#import mmaction

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold

from google.colab import drive
drive.mount('/content/drive')

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
USE_CUDA = True
CUDA = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
if CUDA:
    print('run on %s' % device)

In [ ]:
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
# You can find more details at https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md

In [ ]:
!git clone https://github.com/facebookresearch/slowfast

In [ ]:
%cd slowfast
!python setup.py build develop

In [3]:
!export PYTHONPATH=/path/to/SlowFast/slowfast:$PYTHONPATH

In [1]:
%cd slowfast

/content/slowfast


In [11]:
%ls

build/              GETTING_STARTED.md  projects/  slowfast.egg-info/
CODE_OF_CONDUCT.md  INSTALL.md          README.md  tools/
configs/            LICENSE             setup.cfg  VISUALIZATION_TOOLS.md
CONTRIBUTING.md     linter.sh*          setup.py
demo/               MODEL_ZOO.md        slowfast/


In [17]:
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
cfg = get_cfg()
cfg.merge_from_file('./configs/Kinetics/c2/SLOWFAST_4x16_R50.yaml')
cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/pyslowfast/model_zoo/kinetics400/SLOWFAST_4x16_R50.pkl'
cfg.NUM_GPUS = 1
model = build_model(cfg)

In [19]:
model

SlowFast(
  (s1): VideoModelStem(
    (pathway0_stem): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=[1, 7, 7], stride=[1, 2, 2], padding=[0, 3, 3], bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (pathway1_stem): ResNetBasicStem(
      (conv): Conv3d(3, 8, kernel_size=[5, 7, 7], stride=[1, 2, 2], padding=[2, 3, 3], bias=False)
      (bn): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
  )
  (s1_fuse): FuseFastToSlow(
    (conv_f2s): Conv3d(8, 16, kernel_size=[5, 1, 1], stride=[8, 1, 1], padding=[2, 0, 0], bias=False)
    (bn): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine

In [23]:
for param in model.parameters():
    param.requires_grad = False

n_inputs = model.head.projection.in_features
model.head.projection = nn.Linear(n_inputs, 10)

In [24]:
model

SlowFast(
  (s1): VideoModelStem(
    (pathway0_stem): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=[1, 7, 7], stride=[1, 2, 2], padding=[0, 3, 3], bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (pathway1_stem): ResNetBasicStem(
      (conv): Conv3d(3, 8, kernel_size=[5, 7, 7], stride=[1, 2, 2], padding=[2, 3, 3], bias=False)
      (bn): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
  )
  (s1_fuse): FuseFastToSlow(
    (conv_f2s): Conv3d(8, 16, kernel_size=[5, 1, 1], stride=[8, 1, 1], padding=[2, 0, 0], bias=False)
    (bn): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine

In [ ]:
print("Modelli Trovati") 
%ls drive/MyDrive/*.pth 

Modelli Trovati
drive/MyDrive/_SlowFast_SGD_train_e10.pth
drive/MyDrive/_SlowFast_SGD_train_e5.pth
drive/MyDrive/_SlowFast_SGD_train_e6.pth
drive/MyDrive/_SlowFast_SGD_train_e7.pth
drive/MyDrive/_SlowFast_SGD_train_e8.pth
drive/MyDrive/_SlowFast_SGD_train_e9.pth
drive/MyDrive/_SlowFast_train_e4.pth
drive/MyDrive/_VideoResNet_SGD_train_e3.pth
drive/MyDrive/_VideoResNet_SGD_train_e4.pth
drive/MyDrive/_VideoResNet_SGD_train_e5.pth
drive/MyDrive/_VideoResNet_train_e0.pth
drive/MyDrive/_VideoResNet_train_e1.pth
drive/MyDrive/_VideoResNet_train_e2.pth


In [ ]:
models = ['drive/MyDrive/_SlowFast_SGD_train_e10.pth','drive/MyDrive/_VideoResNet_SGD_train_e5.pth']
for model in models: 
  checkpoint_init = torch.load(model, map_location=device)
  starting_epoch = checkpoint_init['epoch'] + 1 
  tr_loss, val_loss = checkpoint_init['losses']
  tr_accs, val_accs = checkpoint_init['accs']
  print("model name: " + model)
  plt.plot(tr_accs)
  plt.plot(val_accs)
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  plt.close()

  plt.plot(tr_loss)
  plt.plot(val_loss)
  plt.title('model loss')
  
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  plt.close()
  